<a href="https://colab.research.google.com/github/ahmedmalaa/evaluating-generative-models/blob/boris/toy_metric_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function imports

In [1]:
from metrics.prdc import compute_prdc
import numpy as np
import matplotlib.pyplot as plt
import torch

from representations.OneClass import * 
from metrics.evaluation import *

nearest_k = 5
params  = dict({"rep_dim": None, 
                "num_layers": 2, 
                "num_hidden": 200, 
                "activation": "ReLU",
                "dropout_prob": 0.5, 
                "dropout_active": False,
                "train_prop" : 1,
                "epochs" : 100,
                "warm_up_epochs" : 10,
                "lr" : 1e-3,
                "weight_decay" : 1e-2,
                "LossFn": "SoftBoundary"})   

hyperparams = dict({"Radius": 1, "nu": 1e-2})



def plot_all(x, res, x_axis):
    print(x_axis)
    if type(res) == type([]):
        plot_legend = False
        res = {'0':res}
    else:
        plot_legend = True
    exp_keys = list(res.keys())
    print(res)
    metric_keys = res[exp_keys[0]][0].keys() 
    for m_key in metric_keys:
        for e_key in exp_keys:
          y = [res[e_key][i][m_key] for i in range(len(x))]
          plt.plot(x, y, label=e_key)
        
        plt.ylabel(m_key)
        plt.ylim(bottom=0)
        plt.xlabel(x_axis) 
        if plot_legend:
            plt.legend()
        plt.show()


def compute_metrics(X,Y, nearest_k = 5, model = None):
    results = compute_prdc(X,Y, nearest_k)
    if model is None:
        #these are fairly arbitrarily chosen
        params["input_dim"] = X.shape[1]
        params["rep_dim"] = X.shape[1]        
        hyperparams["center"] = torch.ones(X.shape[1])
        
        model = OneClassLayer(params=params, hyperparams=hyperparams)
         
        model.fit(X,verbosity=False)

    X_out = model(torch.tensor(X).float()).float().detach().numpy()
    Y_out = model(torch.tensor(Y).float()).float().detach().numpy()
    
    alphas, alpha_precision_curve, beta_coverage_curve, Delta_precision_alpha, Delta_coverage_beta, (thresholds, authen) = compute_alpha_precision(X_out, Y_out, model.c)
    results['Dpa'] = Delta_precision_alpha
    results['Dcb'] = Delta_coverage_beta
    results['mean_aut'] = np.mean(authen)
    return results, model






In all experiments, X denotes original data and Y synthetic data.

# Translation
Three experiments



1.   $X \sim N(0,I)$, 
$Y \sim N(\mu,I)$
2.   Same as 1. but with one outlier in Y at (1,1,...,1)
3.   Same as 1. but with one outlier in X at (1,1,...,1)


Precision recall is not robust to outliers, even if it's just a single outlier.









In [ ]:
def translation_test(d=64, n=1000, step_size=0.1):
    X = np.random.randn(n,d)
    Y_0 = np.random.randn(n,d)

    X_outlier = X.copy()
    X_outlier[-1] = np.ones(d)

    res = []
    res_outr = []
    res_outf = []

    # translation
    mus = np.arange(-1,1+step_size,step_size)
    model = None
    for i, mu in enumerate(mus):
        Y = Y_0 + mu
        res_, model = compute_metrics(X,Y, model=model)
        res.append(res_)

    model = None
    for i, mu in enumerate(mus):
        Y = Y_0 + mu
        res_, model = compute_metrics(X_outlier,Y, model=model)
        res_outr.append(res_)    
    
    model = None
    for i, mu in enumerate(mus):
        Y_outlier = Y_0 + mu
        Y_outlier[-1] = np.ones(d)
        res_, model = compute_metrics(X, Y_outlier, model = model)
        res_outf.append(res_)

    res = {'No outlier':res, 'Real outlier':res_outr, 'Fake outlier':res_outf}
    plot_all(mus, res, r'$\mu$')


translation_test(n=10000, d=64)

# Mode dropping
X is mixture of Gaussian nodes. For Y, all nodes except zeroth are dropped slowly. Either simultaneously (all nodes except zeroth node are dropped), or sequentially. 

$\Delta P_\alpha$ is better than benchmarks, as it decreases steadier and is thus more sensitive to small imbalances in mode generation. 

In [ ]:
def mode_drop_test(n=1000, d=1, num_nodes=10, dist_between_nodes=4):
    
    # random assignment
    assignment = np.floor(np.random.rand(n)*num_nodes)
    means = np.zeros((n,d))
    means[:,0] = assignment * dist_between_nodes
    X = np.random.randn(n,d) + means
    Y_0 = np.random.randn(n,d)

    res_md_sim = []
    step_size = 0.04
    
    #simultaneous dropping
    p_mode_drops = np.arange(0,1+step_size,step_size)
    model = None

    assignment_Y_ = np.floor(np.random.rand(n)*num_nodes)

    for p_mode_drop in p_mode_drops:
        mode_drop = np.random.rand(n)<p_mode_drop
        # if mode_drop, assign to first Gaussian
        assignment_Y = assignment_Y_ * (1-mode_drop)
        means = np.zeros((n,d))
        means[:,0] = assignment_Y * dist_between_nodes
        Y = Y_0 + means
        res_, model = compute_metrics(X,Y, model=model)
        res_md_sim.append(res_)
 
    # sequential dropping    
    res_md_seq = []
    
    assignment_Y = np.sort(np.floor(np.random.rand(n)*num_nodes))
    len_nonzero = (assignment_Y != 0).sum()
    
    for p_mode_drop in p_mode_drops:
         if p_mode_drop>1e-8:
           assignment_Y[-round(len_nonzero*p_mode_drop):] = 0
         
         means = np.zeros((n,d))
         means[:,0] = assignment_Y * dist_between_nodes
         Y = Y_0 + means
         res_, _ = compute_metrics(X,Y,model=model)
         res_md_seq.append(res_)
    
    print(X.dtype, Y.dtype)
    res = {}
    res['Simultaneous'] = res_md_sim
    res['Sequential'] = res_md_seq
    plot_all(p_mode_drops, res, r'p')

mode_drop_test(n=10000, d=1, num_nodes=10, dist_between_nodes = 4)

# Mode resolution
X consists of a mixture of standard Gaussians at $\pm 0.5\mu$, Y is a Gaussian with the same variance as $X$ centred at 0. This tests whether the metric can distinguish different modes well, i.e. two modes in the original data vs one mode in the synthetic data.

Both $\Delta P_\alpha$ and $\Delta C_\beta$ go down quicker than the other metrics, which means the metric has a higher resolution

In [ ]:
def mode_resolution_test(n=1000,d=1):
    step_size = 0.1
    dists = np.arange(0,5+step_size,step_size)
    res_resolution = []
    for dist in dists:
        mus = dist*((np.random.rand(n)>0.5)-0.5)
        X = ((np.random.randn(n,d)).T + mus).T
        std = np.std(X,axis=0)
        Y = std * np.random.randn(n,d)
        res_, _ = compute_metrics(X,Y, model=None)
        res_resolution.append(res_)
      # mode resolution

    plot_all(dists, res_resolution, 'distance')

mode_resolution_test(n=10000,d=1)

# Variance test
$X\sim N(0,I)$

$Y\sim (N0, \sigma I)$

$\Delta P_\alpha$ shows the clearest peak at $\sigma=1$.

In [ ]:
def variance_test(n=1000, d=64):
    X = np.random.randn(n,d)
    step_size = 0.05
    stds = np.arange(0,2+step_size,step_size)#np.exp(np.arange(-2,1,step_size))
    res = []
    model = None
    for std in stds:
        Y = np.random.randn(n,d)*std
        res_, model = compute_metrics(X,Y, model=model)
        res.append(res_)
    
    plot_all(stds, res, 'std')
    
variance_test(n=10000, d=2)

# Outlier proportion
Metrics as a funtion of outliers.
X standard normal, Y contain $p_{out}$ proportion of outlier points (defined as points on the hypersphere with radius 5).

Our metrics don't do too well on this test.

In [ ]:
def outlier_proportion_test(n=1000, d=64):
    X = np.random.randn(n,d)
    X_outlier = np.random.randn(n,d)
    X_outlier = (X_outlier.T/np.sum(X_outlier**2,axis=1)).T*5
    
    Y = np.random.randn(n,d)
    Y_outlier = np.random.randn(n,d)
    Y_outlier = (Y_outlier.T/np.sum(Y_outlier**2,axis=1)).T *5
    
    
    res_outr = []
    res_outf = []

    # Outlier proportion
    step_size = 0.02
    p_outlier = np.arange(0,0.5+step_size,step_size)
    model = None

    for p in p_outlier:
        c = int(p*n)
        X_p = np.concatenate((X_outlier[:c],X[c:]),axis=0)
        res_, _ = compute_metrics(X_p, Y)
        res_outr.append(res_)    

    for p in p_outlier:
        c = int(p*n)
        Y_p = np.concatenate((Y_outlier[:c],Y[c:]),axis=0)
        res_, model = compute_metrics(X, Y, model=model)
        res_outf.append(res_)    
        
    res = {'Real outliers':res_outr, 'Fake outliers':res_outf}
    plot_all(p_outlier, res, r'$p_{out}$') 
    

outlier_proportion_test(n=10000)

# Autenticity test

Test in which proportion of original data is exactly included in synthetic data

In [ ]:
def copy_test(n=1000, d=64):
    X = np.random.randn(n,d)
    Y = np.random.randn(n,d)
    
    # Copied proportion
    step_size = 0.1
    p_copied = np.arange(0,1+step_size,step_size)
    
    model = None
    res = []

    for p in p_copied:
        c = int(p*n)
        Y_p = np.concatenate((X[:c],Y[c:]),axis=0)
        res_, model = compute_metrics(X, Y, model=model)
        res.append(res_)    
        
    res = {'autenticity':res}
    plot_all(p_copied, res, r'$p_{copied}$') 



copy_test(n=10000, d=8)

Test in which all data is copied but random noise is added of different levels.

In [ ]:
def copy_with_noise_test(n=1000, d=64):
    X = np.random.randn(n,d)
    
    # Outlier proportion
    step_size = 0.1
    noise_level = np.arange(0,1+step_size,step_size)
    
    model = None
    res = []

    for std in noise_levels:
        Y = X +np.random.randn(n,d,) * std
        res_, model = compute_metrics(X, Y, model=model)
        res.append(res_)    
        
    res = {'autenticity':res}
    plot_all(noise_levels, res, r'$std_noise$') 

copy_with_noise_test(n=10000, d=8)